In [1]:
import os,sys
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *
from tqdm import tqdm
from collections import Counter
import pandas as pd
import shutil
import pickle
import gc, math
from math import log, floor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import *
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
max_words = 332 #dict size (tri-gram總數量)
max_length = 106 #第一轉折點 tri-gram num (最靠近outlier數字的斜率變化轉折點) 超過此數量的hkl拿掉
min_length = 33 #最小的轉折點(第三個轉折點) #Q1

In [3]:
root_dir = "./data/Hooklog/trace_picked_fam/"
fam_dirs = next(os.walk(root_dir))[1]
trainX = []
trainY = []
for fam in tqdm(fam_dirs):
    in_directory = root_dir + fam + '/'
    hl_list = next(os.walk(in_directory))[2] # get all filenames in the in_directory
    hl_list = [os.path.join(in_directory, f) for f in hl_list] # filepathname list
    #hl_list = list(filter(lambda f: f.endswith(".enc.npy"), hl_list)) # in case some non-hooklog file in the folder
    hl_list = list(filter(lambda f: f.endswith(".norm.npy"), hl_list)) #load norm data
    for npy in hl_list:
        tri_enc = np.load(npy)
        if len(tri_enc)>max_length:
            tri_enc = tri_enc[:max_length]
        if len(tri_enc)<min_length:
            continue
        trainY.append(fam)
        trainX.append(tri_enc)
print(len(trainY),len(trainX))

100%|██████████| 14/14 [00:00<00:00, 28.11it/s]

2416 2416


In [25]:
train_X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [4]:
fam_code = {}
for i,v in enumerate(list(set(trainY))):
    fam_code[v] = i
# fam_code = pickle.load(file=open(root_dir+"fam_dict_enc.pkl",'rb'))
train_Y = []
for fam in trainY:
    train_Y.append(fam_code[fam])
train_Y = np.array(train_Y)
pickle.dump(obj=fam_code,file=open(root_dir+"fam_dict_enc.pkl",'wb'))
train_Y

array([ 8,  8,  8, ..., 10, 10, 10])

In [5]:
trainX = [list(x) for x in trainX]
# trainX

In [6]:
train_X = tf.keras.preprocessing.sequence.pad_sequences(trainX,maxlen=max_length,padding='post',value=0,dtype='float32')
print(train_X.shape, train_Y.shape)

(2416, 106) (2416,)


In [38]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [39]:
X_train, y_train, X_test, y_test = split_valid_set(train_X, train_Y, 0.9)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
assert len(set(y_test)) == len(set(y_train))
print(set(y_test))

(2174, 106) (2174,) (242, 106) (242,)
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}


In [7]:
# train_X = np.expand_dims(train_X,axis=-1)

In [44]:
X_train = np.expand_dims(X_train,axis=-1)
X_test = np.expand_dims(X_test,axis=-1)
print(X_train.shape,X_test.shape)

(2174, 106, 1) (242, 106, 1)


**for Mike ML data**

In [8]:
def convert_onehot(X_train=X_train):
    """
    input: padded numpy array encode by int
    output: one hot list(samples) of list(tri-gram terms)
    """
    X_train_li = []
    for arr_npy in tqdm(X_train):
        arr_li = []
        for element in arr_npy:
            init_npy = np.zeros((max_words,))
            one_hot = init_npy
            if element ==0:
                arr_li.append(init_npy)
                continue
            one_hot[element] = 1
            arr_li.append(one_hot)
        X_train_li.append(arr_li)
    return X_train_li
X_test_li = convert_onehot(X_test)
X_train_li = convert_onehot(X_train)

100%|██████████| 2174/2174 [00:00<00:00, 3331.94it/s]


In [9]:
def conver2df(X_test_li=X_test_li, y_test=y_test):
    """
    Input: output of convert_onehot function &　it;s corresponding label
    Output: df
    """
    col = ['word_'+str(i) for i in range(max_length)]
    col.append('LABEL')
    df = pd.DataFrame(columns=col)
    for one_hot_li, label in tqdm(zip(X_test_li, y_test)):
        temp = [list(element) for element in one_hot_li]
        temp.extend([label])
        temp = pd.Series(temp,col)
        df = df.append(temp,ignore_index=True)
    return df
test_df = conver2df(X_test_li,y_test)
train_df = conver2df(X_train_li,y_train)

242it [00:02, 80.99it/s]
2174it [00:35, 60.72it/s]


In [10]:
test_df.to_csv("./data/Hooklog/trace_picked_fam/Bi-testing_onehot.csv",index=False)
train_df.to_csv("./data/Hooklog/trace_picked_fam/Bi-training_onehot.csv",index=False)
test_df = pd.read_csv("./data/Hooklog/trace_picked_fam/Bi-testing_onehot.csv")
test_df #check saving

,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,...,word_97,word_98,word_99,word_100,word_101,word_102,word_103,word_104,word_105,LABEL
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0

## Build Model

hyper-parameters

In [9]:
max_length = train_X.shape[-1]
cat_num = train_Y.shape[0]

In [10]:
emb_dim = 128#validXr.shape[-1] 
max_length = X_test.shape[-1] # max sequence length
cat_num = y_test.shape[0]
do = 0.1
vocabulary_size = max_words+1
hidden_dim = 64
EPOCHS=2000
# traina = True #改
batch_size = 128 #改

In [73]:
# init_w = tensorflow.keras.initializers.Constant(value=0.9) #portyion=0.6, w=0.9, b = 0.8-0.85 (0.83從0開始)
init_w = tensorflow.keras.initializers.RandomNormal(mean=0.9,stddev=0.06)
# init_b = tensorflow.keras.initializers.Constant(value=0.84) #w=1 ; b=0.499, portion=1
init_b = tensorflow.keras.initializers.RandomNormal(mean=0.7,stddev=0.2)
def onezero(x):
    portion = 1#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0],reshuffle_each_iteration=True).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)

In [17]:
class test_model(Model):
    def __init__(self):
        super(test_model, self).__init__()
        self.emb = Embedding(vocabulary_size+1, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn = LSTM(hidden_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.bn2 = BatchNormalization(name='bn2')
        self.att = Attention(name='selfatt')
        self.rnn2 = GRU(int(hidden_dim/2),name='common_extract2'
                      ,trainable=True)
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        
        self.cnn = Conv1D(int(hidden_dim/2),strides=2,kernel_size=4)
#         self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
        self.out = Dense(max(train_Y)+1,activation='softmax',name='clf')
        self.flat = Flatten()
    def call(self,x):
        #x1 = self.emb(x)
        x = self.cnn(x)
        x = self.rnn(x)
        x = self.bn1(x)
        #x = self.att([x,x1,x1])
        #x = self.att([x,x,x])
        x = self.rnn2(x)
        x = self.bn2(x)
        #x = self.flat(x)
        x = self.dn(x)
        y2 = self.out(x)
        return y2
model = test_model()
# batch_size = 32
try:
    train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(X_train.shape[0]).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(batch_size)
except NameError:
    train_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).shuffle(train_X.shape[0]).batch(batch_size)
    valid_ds = tf.data.Dataset.from_tensor_slices((train_X,train_Y)).batch(batch_size)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Nadam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') 
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(x,yc):
    with tf.GradientTape() as tape:
        pred_cat = model(x) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss = loss_object(yc, pred_cat)
    trainable_variable = model.trainable_variables
    gradients = tape.gradient(loss,trainable_variable)
    optimizer.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_accuracy(yc, pred_cat) #acc_rate

@tf.function
def test_step(x,yc):
    pred_cat = model(x) #weight: pred_imp ; binary:pred_imp2
    t_loss = loss_object2(yc, pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)


for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels) #with restore words
    
    template = 'Epoch {}, Total Loss: {}, Accuracy Rate: {:5.2f}%,\
            Test_Total_Loss: {}, Test_Accuracy_Rate: {:5.2f}%'
    print(template.format(epoch+1,train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100
                        ))

    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 2.6321074962615967, Accuracy Rate:  9.85%,            Test_Total_Loss: 2.6222646236419678, Test_Accuracy_Rate: 10.80%
Epoch 2, Total Loss: 2.620380401611328, Accuracy Rate: 12.21%,            Test_Total_Loss: 2.6071951389312744, Test_Accuracy_Rate: 12.04%
Epoch 3, Total Loss: 2.6324777603149414, Accuracy Rate: 11.59%,            Test_Total_Loss: 2.614319324493408, Test_Accuracy_Rate: 11.18%
Epoch 4, Total Loss: 2.605168104171753, Accuracy Rate: 14.07%,            Test_Total_Loss: 2.597059726715088, Test_Accuracy_Rate: 13.66%
Epoch 5, Total Loss: 2.592358350753784, Accuracy Rate: 13.25%,            Test_Total_Loss: 2.5813372135162354, Test_Accuracy_Rate: 13.49%
Epoch 6, Total Loss: 2.5672852993011475, Accuracy Rate: 15.81%,            Test_Total_Loss: 2.542290687561035, Test_Accuracy_Rate: 14.53%
Epoch 7, Total Loss: 2.5320024490356445, Accuracy Rate: 14.49%,            Test_Total_Loss: 2.5214712619781494, Test_Accuracy_Rate: 14.32%
Epoch 8, Total Loss: 2.5178937911

Epoch 61, Total Loss: 2.075007438659668, Accuracy Rate: 33.20%,            Test_Total_Loss: 2.058689594268799, Test_Accuracy_Rate: 35.35%
Epoch 62, Total Loss: 2.067067861557007, Accuracy Rate: 33.73%,            Test_Total_Loss: 2.064682722091675, Test_Accuracy_Rate: 33.77%
Epoch 63, Total Loss: 2.062241315841675, Accuracy Rate: 33.44%,            Test_Total_Loss: 2.053793430328369, Test_Accuracy_Rate: 33.73%
Epoch 64, Total Loss: 2.0644454956054688, Accuracy Rate: 33.03%,            Test_Total_Loss: 2.1141021251678467, Test_Accuracy_Rate: 33.49%
Epoch 65, Total Loss: 2.061877965927124, Accuracy Rate: 34.40%,            Test_Total_Loss: 2.0341010093688965, Test_Accuracy_Rate: 34.40%
Epoch 66, Total Loss: 2.0397214889526367, Accuracy Rate: 34.06%,            Test_Total_Loss: 2.0387449264526367, Test_Accuracy_Rate: 34.85%
Epoch 67, Total Loss: 2.0322370529174805, Accuracy Rate: 34.98%,            Test_Total_Loss: 2.059960126876831, Test_Accuracy_Rate: 34.69%
Epoch 68, Total Loss: 2.0312

Epoch 120, Total Loss: 1.7107726335525513, Accuracy Rate: 44.74%,            Test_Total_Loss: 1.6713680028915405, Test_Accuracy_Rate: 46.65%
Epoch 121, Total Loss: 1.718066930770874, Accuracy Rate: 44.33%,            Test_Total_Loss: 1.6641285419464111, Test_Accuracy_Rate: 46.27%
Epoch 122, Total Loss: 1.696630597114563, Accuracy Rate: 46.23%,            Test_Total_Loss: 1.67356538772583, Test_Accuracy_Rate: 45.74%
Epoch 123, Total Loss: 1.6792237758636475, Accuracy Rate: 45.65%,            Test_Total_Loss: 1.6663469076156616, Test_Accuracy_Rate: 46.07%
Epoch 124, Total Loss: 1.6972556114196777, Accuracy Rate: 44.78%,            Test_Total_Loss: 1.680889368057251, Test_Accuracy_Rate: 46.15%
Epoch 125, Total Loss: 1.696319818496704, Accuracy Rate: 44.87%,            Test_Total_Loss: 1.6602802276611328, Test_Accuracy_Rate: 46.23%
Epoch 126, Total Loss: 1.7153617143630981, Accuracy Rate: 44.16%,            Test_Total_Loss: 1.6510720252990723, Test_Accuracy_Rate: 46.61%
Epoch 127, Total Lo

Epoch 179, Total Loss: 1.3857065439224243, Accuracy Rate: 54.01%,            Test_Total_Loss: 1.3944543600082397, Test_Accuracy_Rate: 53.85%
Epoch 180, Total Loss: 1.3886293172836304, Accuracy Rate: 54.10%,            Test_Total_Loss: 1.3517884016036987, Test_Accuracy_Rate: 55.92%
Epoch 181, Total Loss: 1.3982362747192383, Accuracy Rate: 54.64%,            Test_Total_Loss: 1.3426121473312378, Test_Accuracy_Rate: 56.04%
Epoch 182, Total Loss: 1.3617717027664185, Accuracy Rate: 55.79%,            Test_Total_Loss: 1.3402072191238403, Test_Accuracy_Rate: 56.71%
Epoch 183, Total Loss: 1.3769100904464722, Accuracy Rate: 54.84%,            Test_Total_Loss: 1.6021952629089355, Test_Accuracy_Rate: 48.51%
Epoch 184, Total Loss: 1.4388985633850098, Accuracy Rate: 52.73%,            Test_Total_Loss: 1.3242002725601196, Test_Accuracy_Rate: 56.37%
Epoch 185, Total Loss: 1.358546257019043, Accuracy Rate: 54.59%,            Test_Total_Loss: 1.3128184080123901, Test_Accuracy_Rate: 56.37%
Epoch 186, Tot

Epoch 238, Total Loss: 1.1771929264068604, Accuracy Rate: 60.43%,            Test_Total_Loss: 1.1216963529586792, Test_Accuracy_Rate: 61.88%
Epoch 239, Total Loss: 1.1879984140396118, Accuracy Rate: 59.35%,            Test_Total_Loss: 1.1325260400772095, Test_Accuracy_Rate: 61.67%
Epoch 240, Total Loss: 1.168059229850769, Accuracy Rate: 60.39%,            Test_Total_Loss: 1.1001497507095337, Test_Accuracy_Rate: 62.62%
Epoch 241, Total Loss: 1.1540008783340454, Accuracy Rate: 60.64%,            Test_Total_Loss: 1.1116737127304077, Test_Accuracy_Rate: 62.17%
Epoch 242, Total Loss: 1.1602498292922974, Accuracy Rate: 60.10%,            Test_Total_Loss: 1.1033653020858765, Test_Accuracy_Rate: 61.92%
Epoch 243, Total Loss: 1.1621646881103516, Accuracy Rate: 60.51%,            Test_Total_Loss: 1.1333531141281128, Test_Accuracy_Rate: 61.55%
Epoch 244, Total Loss: 1.2504947185516357, Accuracy Rate: 57.82%,            Test_Total_Loss: 1.1314278841018677, Test_Accuracy_Rate: 62.09%
Epoch 245, Tot

...

Epoch 1759, Total Loss: 0.8363129496574402, Accuracy Rate: 66.06%,            Test_Total_Loss: 0.807265043258667, Test_Accuracy_Rate: 68.42%
Epoch 1760, Total Loss: 0.8404488563537598, Accuracy Rate: 66.06%,            Test_Total_Loss: 0.8142654299736023, Test_Accuracy_Rate: 68.05%
Epoch 1761, Total Loss: 0.8590822815895081, Accuracy Rate: 65.40%,            Test_Total_Loss: 0.8235123753547668, Test_Accuracy_Rate: 68.05%
Epoch 1762, Total Loss: 0.8610759377479553, Accuracy Rate: 65.81%,            Test_Total_Loss: 0.824224591255188, Test_Accuracy_Rate: 68.05%
Epoch 1763, Total Loss: 0.8506776690483093, Accuracy Rate: 65.60%,            Test_Total_Loss: 0.8130752444267273, Test_Accuracy_Rate: 68.38%
Epoch 1764, Total Loss: 0.8413125872612, Accuracy Rate: 66.18%,            Test_Total_Loss: 0.814016580581665, Test_Accuracy_Rate: 68.00%
Epoch 1765, Total Loss: 0.8478116393089294, Accuracy Rate: 65.52%,            Test_Total_Loss: 0.8160305619239807, Test_Accuracy_Rate: 67.96%
Epoch 1766, 

Epoch 1817, Total Loss: 0.9032789468765259, Accuracy Rate: 64.53%,            Test_Total_Loss: 0.8406438827514648, Test_Accuracy_Rate: 67.34%
Epoch 1818, Total Loss: 0.9015021324157715, Accuracy Rate: 65.07%,            Test_Total_Loss: 0.8563469052314758, Test_Accuracy_Rate: 67.18%
Epoch 1819, Total Loss: 0.8890103697776794, Accuracy Rate: 65.56%,            Test_Total_Loss: 0.8370493054389954, Test_Accuracy_Rate: 67.63%
Epoch 1820, Total Loss: 0.8642792105674744, Accuracy Rate: 66.31%,            Test_Total_Loss: 0.8323613405227661, Test_Accuracy_Rate: 67.80%
Epoch 1821, Total Loss: 0.868111789226532, Accuracy Rate: 65.85%,            Test_Total_Loss: 0.8217044472694397, Test_Accuracy_Rate: 68.17%
Epoch 1822, Total Loss: 0.853160560131073, Accuracy Rate: 66.23%,            Test_Total_Loss: 0.8172500133514404, Test_Accuracy_Rate: 68.05%
Epoch 1823, Total Loss: 0.8453687429428101, Accuracy Rate: 65.89%,            Test_Total_Loss: 0.8146979212760925, Test_Accuracy_Rate: 68.17%
Epoch 18

Epoch 1875, Total Loss: 0.8367888927459717, Accuracy Rate: 65.98%,            Test_Total_Loss: 0.8052323460578918, Test_Accuracy_Rate: 68.63%
Epoch 1876, Total Loss: 0.8408229351043701, Accuracy Rate: 65.56%,            Test_Total_Loss: 0.8061327338218689, Test_Accuracy_Rate: 68.58%
Epoch 1877, Total Loss: 0.8416199684143066, Accuracy Rate: 65.89%,            Test_Total_Loss: 0.8078007698059082, Test_Accuracy_Rate: 68.05%
Epoch 1878, Total Loss: 0.8343505859375, Accuracy Rate: 65.65%,            Test_Total_Loss: 0.8036569952964783, Test_Accuracy_Rate: 68.46%
Epoch 1879, Total Loss: 0.8335385322570801, Accuracy Rate: 65.69%,            Test_Total_Loss: 0.8052473068237305, Test_Accuracy_Rate: 68.50%
Epoch 1880, Total Loss: 0.8358489871025085, Accuracy Rate: 66.18%,            Test_Total_Loss: 0.8043157458305359, Test_Accuracy_Rate: 68.50%
Epoch 1881, Total Loss: 0.8346988558769226, Accuracy Rate: 66.39%,            Test_Total_Loss: 0.8086549639701843, Test_Accuracy_Rate: 68.21%
Epoch 188

Epoch 1933, Total Loss: 0.9080163836479187, Accuracy Rate: 65.23%,            Test_Total_Loss: 0.8825581073760986, Test_Accuracy_Rate: 66.68%
Epoch 1934, Total Loss: 1.2252581119537354, Accuracy Rate: 60.26%,            Test_Total_Loss: 1.1038333177566528, Test_Accuracy_Rate: 62.33%
Epoch 1935, Total Loss: 1.1020634174346924, Accuracy Rate: 61.09%,            Test_Total_Loss: 1.0933376550674438, Test_Accuracy_Rate: 61.51%
Epoch 1936, Total Loss: 1.1209312677383423, Accuracy Rate: 59.81%,            Test_Total_Loss: 1.048132300376892, Test_Accuracy_Rate: 62.96%
Epoch 1937, Total Loss: 1.1245681047439575, Accuracy Rate: 60.72%,            Test_Total_Loss: 1.0280051231384277, Test_Accuracy_Rate: 63.08%
Epoch 1938, Total Loss: 1.0335384607315063, Accuracy Rate: 62.38%,            Test_Total_Loss: 0.9520912170410156, Test_Accuracy_Rate: 64.74%
Epoch 1939, Total Loss: 0.985898494720459, Accuracy Rate: 63.91%,            Test_Total_Loss: 0.887610673904419, Test_Accuracy_Rate: 66.56%
Epoch 194

Epoch 1991, Total Loss: 0.8287538886070251, Accuracy Rate: 66.39%,            Test_Total_Loss: 0.8019277453422546, Test_Accuracy_Rate: 68.71%
Epoch 1992, Total Loss: 0.8290338516235352, Accuracy Rate: 66.47%,            Test_Total_Loss: 0.8012670874595642, Test_Accuracy_Rate: 68.54%
Epoch 1993, Total Loss: 0.8282462358474731, Accuracy Rate: 66.10%,            Test_Total_Loss: 0.8035615682601929, Test_Accuracy_Rate: 68.46%
Epoch 1994, Total Loss: 0.8294286727905273, Accuracy Rate: 65.94%,            Test_Total_Loss: 0.8011434078216553, Test_Accuracy_Rate: 68.25%
Epoch 1995, Total Loss: 0.8278380036354065, Accuracy Rate: 65.98%,            Test_Total_Loss: 0.8006498217582703, Test_Accuracy_Rate: 68.67%
Epoch 1996, Total Loss: 0.82840496301651, Accuracy Rate: 66.51%,            Test_Total_Loss: 0.8032488822937012, Test_Accuracy_Rate: 68.54%
Epoch 1997, Total Loss: 0.8298474550247192, Accuracy Rate: 66.39%,            Test_Total_Loss: 0.8009476065635681, Test_Accuracy_Rate: 68.54%
Epoch 19

In [100]:
class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(vocabulary_size, emb_dim,input_length=max_length
                             ,trainable=True,name='embedding')
        self.rnn1 = GRU(emb_dim,return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        #x = self.att([x1,x1])
        x = self.rnn1(x1)
        x = self.bn1(x)
        x = self.att([x,x1,x1])
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
model1.load_weights(saveP1)
model1(X_train[:2])

(<tf.Tensor: shape=(2, 182, 128), dtype=float32, numpy=
 array([[[ 0.05026103, -0.03759918, -0.0075792 , ..., -0.05715944,
          -0.00929288,  0.07889917],
         [ 0.04546504, -0.01330616, -0.08277178, ...,  0.0956112 ,
           0.01197832,  0.04695564],
         [ 0.05026103, -0.03759918, -0.0075792 , ..., -0.05715944,
          -0.00929288,  0.07889917],
         ...,
         [-0.05260951,  0.04872397, -0.03031672, ...,  0.00519623,
          -0.02559018, -0.03490983],
         [-0.05260951,  0.04872397, -0.03031672, ...,  0.00519623,
          -0.02559018, -0.03490983],
         [-0.05260951,  0.04872397, -0.03031672, ...,  0.00519623,
          -0.02559018, -0.03490983]],
 
        [[-0.03128414,  0.03349531, -0.0571312 , ...,  0.00304027,
           0.00620059, -0.00962268],
         [ 0.05026103, -0.03759918, -0.0075792 , ..., -0.05715944,
          -0.00929288,  0.07889917],
         [ 0.09278463,  0.00644924,  0.01887746, ...,  0.08126906,
          -0.07023067,  0.02

In [101]:
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        #self.rnn2 = Bidirectional(LSTM(int(hidden_dim),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.dn = Dense(int(hidden_dim/4),activation='selu',
                        kernel_initializer=tf.keras.initializers.lecun_normal(), name='dn')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        x = self.dn(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [102]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: shape=(1, 182, 181), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [103]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def word_reproduce1(prediction_int):
    return prediction_int
def word_reproduce2(prediction_int):
    return prediction_int
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

# optimizer1 = tf.keras.optimizers.SGD(learning_rate=0.0044, momentum=0.5, nesterov=True)
optimizer1 = tf.keras.optimizers.Nadam()#Adam(learning_rate=0.0003)

In [104]:
train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [107]:
alpha = 0.0000001#0.000002 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = 1-loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #weight: pred_imp; 0/1binary: pred_imp2 。#pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

    
# word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
# word_idx_pd['ori_word'] = word_idx_pd.index 
# word_idx_pd = word_idx_pd.set_index(["ID"])
# print(word_idx_pd)

ep = -1
@tf.function(experimental_relax_shapes=True)
def test_step_inf(x,yc,epoch):
    #ori_text = getKeysByValues(list(x))
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    #if ep != epoch:
        #ep = epoch
        #x = tf.dtypes.cast(x,dtype=tf.float32)
    pred_imp3_ = tf.squeeze(pred_imp2)
    pred_imp3 = tf.dtypes.cast(pred_imp3_,dtype=tf.int32)
    used_x = tf.math.multiply(x,pred_imp3)
    filter_emb = tf.math.multiply(emb,pred_imp2)
        #word_reproduce1(x)
        #word_reproduce2(used_x)
        #used_text = getKeysByValues(list(used_x))
        #tf.print(used_x)
        #print(emb.numpy(),pred_imp, x,pred_imp3,used_x)
        #int_ori_text.append(x)
        #int_fil_text.append(used_x)
    
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #weight: pred_imp ; binary:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    return x, used_x, pred_cat, emb, filter_emb

In [135]:
EPOCHS = 2000
DateID = '2019122601'

saveP1 = './model/Hooklog/'+DateID+'/model1'
saveP2 = './model/Hooklog/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

int_ori_text =[]
int_fil_text = []
int_label = []
pred_label = []
ori_emb = []
fil_emb = []
epoch_num = []
fam_lab = []
pred_lab = []
# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    epoch = tf.convert_to_tensor(epoch, dtype=tf.int64)
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        ori,fil,pred_cat,emb,filter_emb = test_step_inf(test_text, test_labels,epoch) #with restore words
        ori_emb.append(emb.numpy()) #一個epoch有很多個batches
        fil_emb.append(filter_emb.numpy()) #一個epoch有很多個batches
        epoch_num.append(epoch.numpy()) #一個epoch一個數字
        fam_lab.append(test_labels.numpy())#一個epoch有很多個batches
        pred_lab.append(pred_cat.numpy())#一個epoch有很多個batches
        #print(k2)
        #test_step(test_text, test_labels) #original
    int_ori_text.append(ori.numpy())
    int_fil_text.append(fil.numpy())
    int_label.append(test_labels.numpy()) #true label
    pred_label.append(pred_cat.numpy())

    
    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))
    #int_ori_text.append(word_reproduce1.result().numpy())
    #int_fil_text.append(word_reproduce2.result().numpy())
    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    #word_reproduce1.reset_states()
    #word_reproduce2.reset_states()
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()
pred_lab2 = []
for pred in pred_lab:
    pred_lab2.append(list(np.argmax(pred,axis=-1)))

Epoch 1, Total Loss: 0.8048461675643921, Clf Loss: 0.0, Filter Loss: 0.0, Seq Loss: 0.0, Accuracy Rate: 69.15%, Ones Portion: 0.0,             Test_Total_Loss: 4.283809185028076, Test_Clf_Loss: 4.283809185028076, Test_Filter_Loss: 0.9998735189437866, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 30.38%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2019122601/model1 ./model/Hooklog/2019122601/model2
Epoch 2, Total Loss: 0.8320643305778503, Clf Loss: 0.0, Filter Loss: 0.0, Seq Loss: 0.0, Accuracy Rate: 68.14%, Ones Portion: 0.0,             Test_Total_Loss: 4.283809185028076, Test_Clf_Loss: 4.283809185028076, Test_Filter_Loss: 0.9998735189437866, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 30.38%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/Hooklog/2019122601/model1 ./model/Hooklog/2019122601/model2
Epoch 3, Total Loss: 0.8325566053390503, Clf Loss: 0.0, Filter Loss: 0.0, Seq Loss: 0.0, Accuracy Rate: 68.33%, Ones Portion: 0.0,             Test_Total_Loss: 4.283809

KeyboardInterrupt: 

In [144]:
pred_lab2 = []
for pred in pred_lab:
    pred_lab2.append(list(np.argmax(pred,axis=-1)))
len(pred_lab2)

128

In [279]:
# len(ep)
ep

15

In [283]:
recent_ep = -1
for ep, o_e, f_e, p_l, t_l in zip(epoch_num,ori_emb,fil_emb,pred_lab2,fam_lab): #take put each epoch
    if ep != recent_ep:
        whole_dict_ori = {}
        whole_dict_fil = {}
#         print('EP:',ep)
        recent_ep = ep
        count =  0
    for oe,fe,pl,tl in zip(o_e, f_e, p_l, t_l): #take out each process
        count+=1
        try:
            temp1 = whole_dict_ori[tl] #put by true label
            temp1.append(oe)
            temp2 = whole_dict_fil[tl]
            temp2.append(fe)
            whole_dict_ori[tl] = temp1 #original embedding
            whole_dict_fil[tl] = temp2 #filter embedding
        except KeyError:
            temp1 = [oe]
            temp2 = [fe]
            whole_dict_ori[tl] = temp1 
            whole_dict_fil[tl] = temp2
#     print('samples#:',count)
#             if tl == 2:
#                 print([list(oe),])
#                 break            
#         except AttributeError:
#             temp1 = list([oe,])
#             temp2 = list([fe,])
#             whole_dict_ori[tl] = temp1 #tl==2 有問題@@
#             whole_dict_fil[tl] = temp2#tl==2 有問題@@
#             if tl == 2:
#                 print([list(oe),])
#                 break           
# count

EP: 0
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 1
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 2
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 3
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 4
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 5
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 6
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 7
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples#: 192
samples#: 224
samples#: 237
EP: 8
samples#: 32
samples#: 64
samples#: 96
samples#: 128
samples#: 160
samples

237

In [306]:
np.array(whole_dict_fil[0][1:]).shape

(13, 182, 128)

In [308]:
test = np.expand_dims(whole_dict_fil[0][0],axis=0)
cosine_similarity(test[0],np.array(whole_dict_fil[0][1:])[0])

array([[-0.02495082,  1.        ,  1.        , ..., -0.30626938,
        -0.30626938, -0.30626938],
       [-0.15718652, -0.42860642, -0.42860642, ...,  0.20198658,
         0.20198658,  0.20198658],
       [ 0.13184482,  0.4822547 ,  0.4822547 , ...,  0.04879938,
         0.04879938,  0.04879938],
       ...,
       [-0.10193527,  0.24838135,  0.24838135, ...,  0.10296886,
         0.10296886,  0.10296886],
       [-0.04273163,  0.62445796,  0.62445796, ..., -0.32532468,
        -0.32532468, -0.32532468],
       [ 0.06389179, -0.11722743, -0.11722743, ..., -0.07050402,
        -0.07050402, -0.07050402]], dtype=float32)

In [258]:
whole_dict_ori

{7: [array([[-0.05921727, -0.22285984, -0.28284442, ...,  0.16880853,
           0.02067087,  0.13023372],
         [-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
          -0.22155283,  0.06595514],
         [ 0.03967505,  0.11265814,  0.13778308, ..., -0.16289778,
           0.05585124,  0.5513557 ],
         ...,
         [ 0.25461718, -0.28900287,  0.17536671, ..., -0.01272231,
          -0.0379485 ,  0.0289838 ],
         [ 0.08445907,  0.2207433 , -0.27071974, ..., -0.02107348,
           0.17360754, -0.25596514],
         [-0.00855983, -0.48185816, -0.20542417, ...,  0.1833916 ,
           0.11007628, -0.4651002 ]], dtype=float32),
  array([[-0.05921727, -0.22285984, -0.28284442, ...,  0.16880853,
           0.02067087,  0.13023372],
         [-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
          -0.22155283,  0.06595514],
         [ 0.03967505,  0.11265814,  0.13778308, ..., -0.16289778,
           0.05585124,  0.5513557 ],
         ...,
         [-0.013

In [200]:
whole_dict_ori[tl] = [fe,]
whole_dict_ori[tl].append(fe)
whole_dict_ori[tl]

[array([[-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
         -0.22155283,  0.06595514],
        [ 0.1538219 ,  0.08651822, -0.05850999, ...,  0.17292847,
          0.0122154 , -0.16761358],
        [ 0.10163565,  0.11445903, -0.09826617, ...,  0.22165859,
         -0.40742022,  0.02286944],
        ...,
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903],
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903],
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903]], dtype=float32),
 array([[-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
         -0.22155283,  0.06595514],
        [ 0.1538219 ,  0.08651822, -0.05850999, ...,  0.17292847,
          0.0122154 , -0.16761358],
        [ 0.10163565,  0.11445903, -0.09826617, ...,  0.22165859,
         -0.40742022,  0.02286944],
        ...,
        [-0.01391036,  0.05648555,  0.0

In [173]:
[fe,fe]

[array([[-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
         -0.22155283,  0.06595514],
        [ 0.1538219 ,  0.08651822, -0.05850999, ...,  0.17292847,
          0.0122154 , -0.16761358],
        [ 0.10163565,  0.11445903, -0.09826617, ...,  0.22165859,
         -0.40742022,  0.02286944],
        ...,
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903],
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903],
        [-0.01391036,  0.05648555,  0.05279909, ..., -0.00105697,
         -0.05969607,  0.01120903]], dtype=float32),
 array([[-0.04800555, -0.06131621, -0.07549804, ..., -0.10737687,
         -0.22155283,  0.06595514],
        [ 0.1538219 ,  0.08651822, -0.05850999, ...,  0.17292847,
          0.0122154 , -0.16761358],
        [ 0.10163565,  0.11445903, -0.09826617, ...,  0.22165859,
         -0.40742022,  0.02286944],
        ...,
        [-0.01391036,  0.05648555,  0.0

In [146]:
# pred_lab2 = list(np.argmax(pred_lab,axis=-1))
len(epoch_num)

128

In [156]:
len(ori_emb[0])

32

In [ ]:
ori_emb 
fil_emb 
epoch_num 
fam_lab 
pred_lab2

## Store process

In [89]:
save_dir = './results/Hooklog/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df.to_excel(saveR)
print(saveR)
df

./results/Hooklog/2019122601/losses_metrics.xlsx


,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num


In [154]:
labelID_dict = pickle.load(open("./data/Hooklog/trace_picked_fam/fam_dict_enc.pkl","rb"))
IDlabel_dict = {v: k for k, v in labelID_dict.items()}
word_index = pickle.load(open("./data/Hooklog/trace_picked_fam/tri_dict_enc.pkl","rb"))
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

pred_label = list(np.argmax(pred_label,axis=-1))
def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        key = item
#         print(key)
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

                                            ori_word
ID                                                  
1               (DeleteFile, DeleteFile, DeleteFile)
2                     (CopyFile, CopyFile, CopyFile)
3               (RegOpenKey, RegOpenKey, RegOpenKey)
4            (CloseHandle, CloseHandle, CloseHandle)
5           (RegOpenKey, RegQueryValue, RegCloseKey)
...                                              ...
1373          (CloseHandle, RegSetValue, CreateFile)
1374     (HttpSendRequest, CloseHandle, LoadLibrary)
1375    (LoadLibrary, InternetOpen, InternetConnect)
1376       (RegQueryValue, LoadLibrary, CloseHandle)
1377  (LoadLibrary, CreateRemoteThread, CloseHandle)

[1377 rows x 1 columns]


In [155]:
word_fil_text = []
word_ori_text = []
from tqdm import tqdm
for text in tqdm(int_ori_text): #text = one epoch
    temp = []
    for t in text: #t = one sample
        temp.append(getKeysByValues(list(t)))
    word_ori_text.append(temp)
for text in tqdm(int_fil_text):
    temp = []
    for t in text:
        temp.append(getKeysByValues(list(t)))
    word_fil_text.append(temp)

save_path = save_dir + "original_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_ori_text)
save_path = save_dir + "filterUsed_words.pkl"
pickle.dump(file=open(save_path,'wb'),obj=word_fil_text)
print(save_path)

  0%|          | 0/2000 [00:00<?, ?it/s]


KeyError: 1383

In [127]:
col = ['epoch#','text_id','ori_text','filter_text','label','predict']
df = pd.DataFrame(columns=col)
for epoch,(text_ori,text_fil,labelID,predID) in tqdm(enumerate(zip(word_ori_text,word_fil_text,int_label,pred_label))):
    for idx,(t_o,t_f,label,pred) in enumerate(zip(text_ori,text_fil,labelID,predID)):
        temp = pd.Series([epoch,idx," ".join([str(x) for x in t_o])," ".join([str(x) for x in t_f]),IDlabel_dict[label],IDlabel_dict[pred]],col)
        df = df.append(temp,ignore_index=True)

save_path = save_dir + "caseStudy_result.xlsx"
df.to_excel(save_path,index=False)
print(save_path)
df

2000it [00:11, 174.48it/s]


./results/Hooklog/2019122301/caseStudy_result.xlsx


,epoch#,text_id,ori_text,filter_text,label
0,0,0,"('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...","('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...",domaiq
1,0,1,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...",installcore
2,0,2,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...",browsefox
3,1,0,"('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...","('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...",domaiq
4,1,1,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...",installcore
...,...,...,...,...,...
5995,1998,1,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...",installcore
5996,1998,2,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'RegOpenKey') (...",browsefox
5997,1999,0,"('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...","('RegOpenKey', 'LoadLibrary', 'LoadLibrary') (...",domaiq
5998,1999,1,"('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...","('LoadLibrary', 'LoadLibrary', 'LoadLibrary') ...",installcore
